# Specviz Demonstration Notebook
This notebook demonstrates the Specviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Specviz, can be found here: https://jdaviz.readthedocs.io/en/latest/specviz/

## Setup

In [1]:
import warnings
import tempfile

import astropy.units as u
from astroquery.mast import Observations

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

## Create Specviz via Helper

In [2]:
from jdaviz import Specviz
specviz = Specviz()

## Display Specviz

Note: you probably want to pick one of the three options below depending on preference. Also not the latter two will only work in JupyterLab, not the "classic" notebook interface.

This will show Specviz inline in the notebook

In [3]:
specviz.show()

Application(config='specviz', events=['call_viewer_method', 'close_snackbar_message', 'data_item_remove', 'dat…

## Load a file to Specviz

Here we download a JWST spectrum from [MAST](https://masttest.stsci.edu/) using `astroquery`. 
By default the downloaded file goes to your temp directory, and thus may eventually be 
garbage collected (deleted) by your system. If you would like to have the file permanently, 
simply uncomment the second line below and provide the directory path on your system where 
you would like the file stored.

One other thing to note about retrieving MAST data through astroquery is that it caches
the data by default. It is possible for files to be updated in MAST with more recent calibrations
but remain the same size, in which case your local cached file would not be replaced by the new
version. You can turn off caching by setting `cache=False` in the `download_file` call to
force it to re-download the file if desired.

In [4]:
from glue.core.roi import XRangeROI

data_dir = "/Users/javerbukh/Documents/jdaviz_dev/data/"
file = "jw02732-c1001_t004_miri_ch1-longmediumshort-_x1d.fits"
file2 = "jw02732-c1001_t004_miri_ch2-longmediumshort-_x1d.fits"
specviz.load_spectrum(data_dir + file)
specviz.load_spectrum(data_dir + file2)

specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(7, 8))

In [5]:
# specviz.app.data_collection.subset_groups[0].subsets[1].data.get_selection_definition(format='astropy-regions')
for x in specviz.app.data_collection.subset_groups[0].subsets:
    print(len(x.data.pixel_component_ids))
    print(x.data.get_selection_definition(format='astropy-regions'))

1


ValueError: Range subset state att should be either x or y pixel coordinate

In [ ]:
data_dir = tempfile.gettempdir()
#data_dir = "/User/username/Data/"

fn = "jw02732-o004_t004_miri_ch1-shortmediumlong_x1d.fits"
result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

specviz.load_spectrum(f'{data_dir}/{fn}', "myfile")

## Subset Retrieval
User Task: Select a subset in the viewer above

After defining regions in your spectra, you can retrieve your subsets through different means. To retrieve your subset by name:

In [ ]:
# Returns a version of the whole spectra, with a mask applied
specviz.get_spectra('Subset 1')

Or, if you've defined multiple regions, you can retrieve all defined regions/subsets via:

In [ ]:
specviz.app.get_subsets_from_viewer('spectrum-viewer')

## Screenshot Saving

In [ ]:
specviz.app.get_viewer("spectrum-viewer").figure.save_png()

## Panning/Zooming in Specviz
### Limit Methods
You can use the methods x_limits() and y_limits() to modify the field of view of Specviz. You can provide it a scalar (which assumes the units of the loaded spectra), an Astropy Quantity, or 'auto' to automatically scale.

In [ ]:
specviz.x_limits()
specviz.x_limits(7*u.um,7.2*u.um)
specviz.y_limits('auto', 170.0)

### Disable Scientific Notation
Scientific notation is used in the axes of the viewers by default. To deactivate it, run the following code:

In [ ]:
# axis 1 corresponds to the Y-axis and axis 0 to the X-axis
# fmt can be '0.1e' to set scientific notation or '0.2f' to turn it off
specviz.set_spectrum_tick_format(fmt="0.2f", axis=1)

### Autoscale Methods
You can also quickly return to the default zoom using autoscale_x() and autoscale_y()

In [ ]:
specviz.autoscale_x()
specviz.autoscale_y()

## Extracting Models
If models were added using the model fitting plugin, they can be extracted using the following property

In [ ]:
specviz.fitted_models

If the name of a particular model is known, it can be extracted from the fitted_models property

In [ ]:
specviz.fitted_models['Model']

Alternatively, the following getter can be used

In [ ]:
models = specviz.get_models()
models